# Using CodeGaze

Examples of the codegaze API.


In [1]:
import os
import pandas as pd
from tqdm.notebook import tqdm
from codegaze import CodeBlockParser, codegen, metrics
from codegaze.utils import save_json, load_json


In [2]:
block_parser = CodeBlockParser()
edit_metric = metrics.EditMetric()
ast_metric = metrics.ASTMetric(parser=block_parser)
neural_metric = metrics.NeuralMetric()


## Computing Code Similarity


In [6]:
code = '''import requests
def get_ip():
    """Get my current external IP. Other line. """
    result = requests.get("https://icanhazip.com").text.strip() 
    holder = []
    for row in result:
        holder.append(row)
    return result
'''

code_blocks = block_parser.extract_blocks(code, max_lines_per_block=3)
a = code_blocks[1].text
b = code_blocks[2].text

edit = edit_metric.compute(a, b)
ast = ast_metric.compute(a, b)
neural = neural_metric.compute(a, b)

print(f"Edit distance: {edit}, AST distance: {ast}, Neural distance: {neural}")


Edit distance: 0.13559322033898302, AST distance: 1.0, Neural distance: 0.930390477180481


## Generating Code Completions

- Support for OpenAI models
- Support for HuggingFace models


In [11]:
oai_model = codegen.OpenAICodeGenerator(name="oaimodel", engine="code-cushman-001")
hf_model = codegen.HFCodeGenerator(
    name="salesforce350", model="Salesforce/codegen-350M-multi"
)
oai_completions = oai_model.generate(code, n=2)
hf_completions = hf_model.generate(code, n=2)


# Run and Analyze Experiment Results


In [16]:
from codegaze.experiment import Experiment
from codegaze.datamodel import ExperimentConfig
from codegaze.datasets import HumanEvalDataset
from codegaze.utils import get_dirs, load_json, save_json
import pandas as pd


In [17]:
exp = Experiment("general", data_path="../experiments/data/")
experiments = exp.list_experiments(dataset="humaneval")
experiment = experiments[0]
print("Working on experiment:", experiment.slug)
models = exp.list_models(experiment.dataset, experiment.slug)
model = models[0]
print("Working on model:", model)


Working on experiment: pt1_ml10_t0.2_suff0
Working on model: code-davinci-001


In [18]:
def agg_results(exp_results):
    exp_list = []
    for metric_type in exp_results.keys():
        for metric in exp_results[metric_type].keys():
            exp_list.append(
                {
                    "metric_type": metric_type,
                    "metric": metric,
                    "value": exp_results[metric_type][metric],
                }
            )
    exp_df = pd.DataFrame(exp_list)
    exp_df
    return exp_df

def metrics_data(experiment, model, metric_type):
    metrics, metrics_arr = exp.load_metrics(experiment, model, metric_type=metric_type)
    return metrics, metrics_arr

In [23]:
all_df = pd.DataFrame()
for model in models:
    exp_results = exp.get_aggregate_results(experiment, model)
    exp_df = agg_results(exp_results)
    exp_df["model"] = model
    all_df = pd.concat([all_df, exp_df])
all_df.head(4)


,metric_type,metric,value,model
0,function,pass@1,0.334146,code-davinci-001
1,function,pass@10,0.493902,code-davinci-001
2,function,func_edit,0.273087,code-davinci-001
3,function,func_ast,0.309624,code-davinci-001


In [17]:
import pandas as pd
import json
from dataclasses import asdict 
import matplotlib.pyplot as plt
from codegaze import CodeBlockParser, codegen, metrics
block_parser = CodeBlockParser()


In [6]:
df = pd.read_csv("HumanEval_to_blockify.csv")
df.head(3)

,Unnamed: 0,problem,prompt,ground truth,unit_tests,generation1,generation2
0,141,HumanEval/57,"def monotonic(l: list):\n """"""Return True is...","if l == sorted(l) or l == sorted(l, reverse=Tr...",METADATA = {}\n\n\ndef check(candidate):\n ...,if len(l) <= 1:\n return True\n if l...,return all(l[i] <= l[i+1] for i in range(len(l...
1,42,HumanEval/130,"def tri(n):\n """"""Everyone knows Fibonacci s...",if n == 0:\n return [1]\n my_tri = [...,def check(candidate):\n\n # Check some simp...,if n == 0:\n return [1]\n elif n == ...,if n == 1:\n return [3]\n elif n == ...
2,113,HumanEval/11,from typing import List\n\n\ndef string_xor(a:...,"def xor(i, j):\n if i == j:\n ...","METADATA = {\n 'author': 'jt',\n 'datase...","return ''.join(str(int(x) ^ int(y)) for x, y i...","return ''.join(str(int(x) ^ int(y)) for x, y i..."


In [11]:
def blockify(prompt, field, max_lines_per_block=5): 
    code_blocks = block_parser.extract_blocks(field, max_lines_per_block=max_lines_per_block, skip_parent_block=False) 
    # print( len(code_blocks))
    code_blocks = [asdict(x)["text"] for x in code_blocks] 
    return code_blocks

In [12]:
df["generation1_blocks"] = df.apply( lambda x: blockify(x.prompt, x.generation1), axis=1)
df["generation2_blocks"] = df.apply( lambda x: blockify(x.prompt, x.generation2), axis=1)
df["ground_truth_blocks"] = df.apply( lambda x: blockify(x.prompt, x["ground truth"]), axis=1)

In [20]:
df.head(2) 

,Unnamed: 0,problem,prompt,ground truth,unit_tests,generation1,generation2,generation1_blocks,generation2_blocks,ground_truth_blocks
0,141,HumanEval/57,"def monotonic(l: list):\n """"""Return True is...","if l == sorted(l) or l == sorted(l, reverse=Tr...",METADATA = {}\n\n\ndef check(candidate):\n ...,if len(l) <= 1:\n return True\n if l...,return all(l[i] <= l[i+1] for i in range(len(l...,"[if len(l) <= 1:\n return True, if l[0]...",[return all(l[i] <= l[i+1] for i in range(len(...,"[if l == sorted(l) or l == sorted(l, reverse=T..."
1,42,HumanEval/130,"def tri(n):\n """"""Everyone knows Fibonacci s...",if n == 0:\n return [1]\n my_tri = [...,def check(candidate):\n\n # Check some simp...,if n == 0:\n return [1]\n elif n == ...,if n == 1:\n return [3]\n elif n == ...,[if n == 0:\n return [1]\n elif n ==...,[if n == 1:\n return [3]\n elif n ==...,"[if n == 0:\n return [1], my_tri = [1, ..."


In [25]:

# all_lens = []
# for row in df.generation1_blocks:
#     all_lens.append(len(row))
#     if len(row) == 1:
#         print(row[0])
#         print("\n********************\n")
# plt.hist(all_lens)

In [15]:
df.to_csv("HumanEval_blockified_max5.csv", index=False)

In [1]:
from codegaze.codegen import HFCodeGenerator, OpenAICodeGenerator

In [2]:
prompt = "def load_json(fisdfle_path):"

In [3]:
hfcodegen = HFCodeGenerator(name="salesforce350", model="Salesforce/codegen-350M-multi") 
oaicodegen = OpenAICodeGenerator(name="openai", model="code-cushman-001")

In [4]:
oai_comp = oaicodegen.generate(prompt, n=10) 

In [7]:
len(oai_comp)

10

In [8]:
hf_gen = hfcodegen.generate(prompt, n=10)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/victordibia/miniconda3/lib/python3.9/site-packages/transformers/models/codegen/modeling_codegen.py:167: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352660876/work/aten/src/ATen/native/TensorCompare.cpp:402.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


In [11]:
for row in hf_gen:
    print( (row.text), "\n*************************\n")


    with open(fisdfle_path) as fisdfle:
        return json.load(fisdfle)


def load_json_file(fisdfle_path):
    with open(fisdfle_path) as fisdfle:
        return json.load(fisdfle)


def load_json_file_with_path(fisdfle_path):
    with open(fisdfle_path) as fisdfle:
        return json.load(fisdfle)


def load_json_file_with_path_and_object(fisdfle_path):
    with open(fisdfle_path) as fisdfle:
        return json.load(fisdfle)


def load_json_file 
*************************


    """
    Loads a json file
    """
    with open(fisdfle_path, 'r') as fisdfle:
        return json.load(fisdfle)


def load_json_file(fisdfle_path):
    """
    Loads a json file
    """
    with open(fisdfle_path, 'r') as fisdfle:
        return json.load(fisdfle)


def load_json_file_with_path(fisdfle_path):
    """
    Loads a json file
    """
    with open(fisdfle_path, 'r') as fisdfle:
        return json.load(fisdfle)


def load_json_file_with_path_as 
*************************


    """
    Loads 

In [70]:
import torch
torch.cuda.is_available()

True

In [12]:
from abc import abstractmethod
from typing import Union
import Levenshtein
import torch

from codegaze import CodeBlockParser
from transformers import AutoTokenizer, AutoModel

# from tokenizers import Tokenizer
from sklearn.metrics.pairwise import cosine_similarity
import logging

logger = logging.getLogger(__name__)


class Metric(object):
    @abstractmethod
    def __init__(self):
        """initialize the metric"""

    @abstractmethod
    def compute(self, a: Union[list[str], str], b: Union[list[str], str]) -> float:
        """Compute distance"""


class NeuralMetric(Metric):
    def __init__(self, hfmodel: str = "microsoft/codebert-base"):
        tokenizer = AutoTokenizer.from_pretrained(hfmodel)
        model = AutoModel.from_pretrained(hfmodel)
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model.to(device)

    def compute(self, a: str, b: str) -> float:
        tokenized_code = tokenizer(
            [a, b], padding=True, truncation=True, return_tensors="pt"
        ).to(device)
        encoded_code = model(tokenized_code.input_ids)["last_hidden_state"]
        emb_a = encoded_code[0].mean(dim=0).detach().cpu().numpy().reshape(1, -1)
        emb_b = encoded_code[1].mean(dim=0).detach().cpu().numpy().reshape(1, -1)

        return float(cosine_similarity(emb_a, emb_b))

In [24]:
nm = NeuralMetric()
a = "def load_json(fisdfle_path):"
b = "def load_csv(file_path):"
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

hfmodel = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(hfmodel)
model = AutoModel.from_pretrained(hfmodel)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

tokenized_code = tokenizer(
            [a, b], padding=True, truncation=True, return_tensors="pt"
        ).to(device)
encoded_code = model(tokenized_code.input_ids)["last_hidden_state"]


0.9728615283966064


In [32]:
emb_a = encoded_code[0].mean(dim=0).detach().cpu().numpy().reshape(1, -1)
emb_b = encoded_code[1].mean(dim=0).detach().cpu().numpy().reshape(1, -1)
print(float(cosine_similarity(emb_a, emb_b)))

0.9728615283966064


In [35]:
g = encoded_code[0].mean(dim=0) 
h = encoded_code[1].mean(dim=0)
output = cos(g, h).item()
float(output)

0.972861647605896